# Project 1 : Vigenere Cipher

## 1. Analysis on the Key Length

By using narrow_down method of VigenereCipher class I've defined in vigenere.py,
I speculated the key length.

It works as follows

1. narrow_down(n, start=0) method returns set of (n keys with offset) that can encrypt plaintext\[start:start+n\] into ciphertext\[start:start+n\] properly.
    e.g) 63 79 81 /10 can encrypt 'THI' into 'OYK', which is the first three letters of the plaintext
    <br>
2. with moderate n(not too big, so it doesn't take too much time), iterate narrow_down start from n to 25, and check if there's keys and offset that appeared in the step 1
    <br>
3. If it exists, it's highly likely that the key length is the 'start'
    e.g) if 63 79 81 /10 has appeared again at start = 4, it's likely that the key size is 4. Since 63, 79, 81 which appeared at position 0, 1, 2 appeared again at position 4, 5, 6
   <br>

In [1]:
from vigenere import VigenereCipher
import time

In [2]:
vc = VigenereCipher()

In [3]:
start = time.time()
analysis_standard = vc.narrow_down(n=5)
end = time.time()
print(f"elapsed time : {end-start}")

elapsed time : 0.13899874687194824


In [4]:
print(len(analysis_standard))
print(list(analysis_standard)[0])

20658
27 24 77 72 31 /4


In [5]:
key_candidates = []
for start in range(5, 26):
    to_be_compared = vc.narrow_down(n=5, start=start)
    for key_offset in to_be_compared:
        if key_offset in analysis_standard:
            key_candidates.append((key_offset, start-1))
            print(f"key candidate : {key_offset}  expected_length : {start}")

key candidate : 52 19 26 00 02 /6  expected_length : 25
key candidate : 52 19 26 28 02 /6  expected_length : 25


Now, it can be inferred that the key length is 25, and the key must have offset 6.
But under that setting, there can be multiple possible keys.
For the next step, I'll build up the key.

## 2. Building Up the Key Candidates

Since offset 6 is known, it's much faster to specify possible key sequence in length n with narrow_down function.
<br>
Since the key length is known, I can build up the key by following steps which are very similar to the method I've used to find out the key length.
<br>
1. set i = 0, moderate n(I've set it 4)
2. get keys by using narrow_down method, narrow_down(n, start=i, offset=6)
3. get keys from the next block(block size=key length = 25) by narrow_down(n, start=i+25, offset=6)
4. compare two keys sets, and specify sequences that are included in both sets, then save it
5. if i haven't reached desired value, increment i and return to step 2
<br>

Only 41 letters are known from the .pdf, but it can be easily inferred that following 6 letters are 'ucture'
With 47 letters, I can validate 22 elements out of 25 key sequences(25+22 letters so 22 letters can be validated)
Thus, I'll build up 22 length key with offset 6.

In [6]:
gram_length = 4
key_length_to_know = 22
iteration = key_length_to_know - gram_length + 1

quad_key_at_start_n = {}
for i in range(iteration):
    quad_key_at_start_n[i] = []

for start in range(iteration):
    standard = vc.narrow_down_offset(n=4, start=start, offset=6)
    to_be_compared = vc.narrow_down_offset(n=4, start=start+25, offset=6)
    for key in to_be_compared:
        if key in standard:
            quad_key_at_start_n[start].append(key)

In [7]:
print(quad_key_at_start_n[0])
print(quad_key_at_start_n[1])

['52 19 26 00', '52 19 26 28']
['19 26 00 02', '19 26 28 02']


From those candidates, two 5 length keys can be built up, 52 19 26 00 02, and 52 19 26 28 02
 I'll build up all possible 22 length keys

In [8]:
def build_up_quad_keys(key_dict):
    build_up = key_dict[0]
    for i in range(1, len(key_dict)):
        new = []
        next_quads = key_dict[i]
        for quad in next_quads:
            for building_key in build_up:
                if building_key[-8:] == quad[:8]:
                    new.append(building_key + quad[-3:])
        build_up = new
    return build_up

In [9]:
built_keys = build_up_quad_keys(quad_key_at_start_n)

In [10]:
print(len(built_keys))
print(built_keys[0])

464
52 19 26 28 02 17 49 65 87 08 94 70 92 88 27 63 39 35 30 31 05 42


There are 464 possible 22 length keys based on the first and the second block.
For the latter step, I'll precompute possible last three sequence of keys and it's as following.

In [11]:
last_three = list(vc.narrow_down_offset(n=3, start=22, offset=6))
print(len(last_three))
print(last_three[0])

60
24 78 65


## 3. Final Key Finding by Decryption

I'll decrypt the first 22 letters of each blocks of ciphertext(125 letters, thus 5 blocks with 25 size exist).
And then look for some plausible outputs.

In [12]:
import re

def keep_with_keyword(keyword, block):
    processed = {}
    for key in block.keys():
        if re.search(keyword, block[key]):
            processed[key]=block[key]
    return processed

In [13]:
block3 = {}
for i in range(len(built_keys)):
    plaintext = vc.decrypt(keys=built_keys[i].split(), offset=6, block=3, block_size=25)
    block3[i] = plaintext
    print(f"{i} : {plaintext}")

0 : INFORMAIIONSECURITYARE
1 : INFORMAIIONSECURITYARE
2 : INFORMATIONSECURITYARE
3 : INFORMATIONSECURITYARE
4 : INFORMAIIONSECURITYARE
5 : INFORMAIIONSECURITYARE
6 : INFORMATIONSECURITYARE
7 : INFORMATIONSECURITYARE
8 : INFORMAIIONSECURITYARE
9 : INFORMAIIONSECURITYARE
10 : INFORMATIONSECURITYARE
11 : INFORMATIONSECURITYARE
12 : INFORMAIIONSECURITYARE
13 : INFORMAIIONSECURITYARE
14 : INFORMATIONSECURITYARE
15 : INFORMATIONSECURITYARE
16 : INFORMAIIONSECURITYJRE
17 : INFORMAIIONSECURITYJRE
18 : INFORMATIONSECURITYJRE
19 : INFORMATIONSECURITYJRE
20 : INFORMAIIONSECURITYJRE
21 : INFORMAIIONSECURITYJRE
22 : INFORMATIONSECURITYJRE
23 : INFORMATIONSECURITYJRE
24 : INFORMAIIONSECURITYHRE
25 : INFORMAIIONSECURITYHRE
26 : INFORMATIONSECURITYHRE
27 : INFORMATIONSECURITYHRE
28 : INFORMAIIONSECURITYHRE
29 : INFORMAIIONSECURITYHRE
30 : INFORMATIONSECURITYHRE
31 : INFORMATIONSECURITYHRE
32 : INFORMAIIONSECURITYJRE
33 : INFORMAIIONSECURITYJRE
34 : INFORMATIONSECURITYJRE
35 : INFORMATIONSECURITYJRE
36

It can be inferred that the plaintext is "Information security ???"
I'll just cross out candidates without "INFORMATIONSECURITY"

In [14]:
block3 = keep_with_keyword("INFORMATIONSECURITY", block3)

In [15]:
len(block3)

232

In [16]:
block4 = {}
for i in range(len(built_keys)):
    plaintext = vc.decrypt(keys=built_keys[i].split(), offset=6, block=4, block_size=25)
    block4[i] = plaintext
    print(f"{i} : {plaintext}")

0 : MOSTPOPOLARKEYWORDSTOC
1 : MOSTPOPOLARKEYWORDSTOC
2 : MOSTPOPULARKEYWORDSTOC
3 : MOSTPOPULARKEYWORDSTOC
4 : MOSTPOPOSARKEYWORDSTOC
5 : MOSTPOPOSARKEYWORDSTOC
6 : MOSTPOPUSARKEYWORDSTOC
7 : MOSTPOPUSARKEYWORDSTOC
8 : MOSTPOPOLARKEYWORDSTOY
9 : MOSTPOPOLARKEYWORDSTOY
10 : MOSTPOPULARKEYWORDSTOY
11 : MOSTPOPULARKEYWORDSTOY
12 : MOSTPOPOSARKEYWORDSTOY
13 : MOSTPOPOSARKEYWORDSTOY
14 : MOSTPOPUSARKEYWORDSTOY
15 : MOSTPOPUSARKEYWORDSTOY
16 : MOSTPOPOLARKEYWORDSVCZ
17 : MOSTPOPOLARKEYWORDSVCZ
18 : MOSTPOPULARKEYWORDSVCZ
19 : MOSTPOPULARKEYWORDSVCZ
20 : MOSTPOPOSARKEYWORDSVCZ
21 : MOSTPOPOSARKEYWORDSVCZ
22 : MOSTPOPUSARKEYWORDSVCZ
23 : MOSTPOPUSARKEYWORDSVCZ
24 : MOSTPOPOLARKEYWORDSZCO
25 : MOSTPOPOLARKEYWORDSZCO
26 : MOSTPOPULARKEYWORDSZCO
27 : MOSTPOPULARKEYWORDSZCO
28 : MOSTPOPOSARKEYWORDSZCO
29 : MOSTPOPOSARKEYWORDSZCO
30 : MOSTPOPUSARKEYWORDSZCO
31 : MOSTPOPUSARKEYWORDSZCO
32 : MOSTPOPOLARKEYWORDSVOC
33 : MOSTPOPOLARKEYWORDSVOC
34 : MOSTPOPULARKEYWORDSVOC
35 : MOSTPOPULARKEYWORDSVOC
36

MOSTPOPULARKEYWORDS make sense the most

In [17]:
block4 = keep_with_keyword("MOSTPOPULARKEYWORDS", block4)

In [18]:
len(block4)

116

In [19]:
block5 = {}
for i in range(len(built_keys)):
    plaintext = vc.decrypt(keys=built_keys[i].split(), offset=6, block=5, block_size=25)
    block5[i] = plaintext
    print(f"{i} : {plaintext}")

0 : TETHENIRELOOKINGPLAINT
1 : TETCENIRELOOKINGPLAINT
2 : TETHENICELOOKINGPLAINT
3 : TETCENICELOOKINGPLAINT
4 : TETHENIRELOOKINGPLAINT
5 : TETCENIRELOOKINGPLAINT
6 : TETHENICELOOKINGPLAINT
7 : TETCENICELOOKINGPLAINT
8 : TETHENIRELOOKINGPLAING
9 : TETCENIRELOOKINGPLAING
10 : TETHENICELOOKINGPLAING
11 : TETCENICELOOKINGPLAING
12 : TETHENIRELOOKINGPLAING
13 : TETCENIRELOOKINGPLAING
14 : TETHENICELOOKINGPLAING
15 : TETCENICELOOKINGPLAING
16 : TETHENIRELOOKINGPLAFUC
17 : TETCENIRELOOKINGPLAFUC
18 : TETHENICELOOKINGPLAFUC
19 : TETCENICELOOKINGPLAFUC
20 : TETHENIRELOOKINGPLAFUC
21 : TETCENIRELOOKINGPLAFUC
22 : TETHENICELOOKINGPLAFUC
23 : TETCENICELOOKINGPLAFUC
24 : TETHENIRELOOKINGPLADUV
25 : TETCENIRELOOKINGPLADUV
26 : TETHENICELOOKINGPLADUV
27 : TETCENICELOOKINGPLADUV
28 : TETHENIRELOOKINGPLADUV
29 : TETCENIRELOOKINGPLADUV
30 : TETHENICELOOKINGPLADUV
31 : TETCENICELOOKINGPLADUV
32 : TETHENIRELOOKINGPLAFNT
33 : TETCENIRELOOKINGPLAFNT
34 : TETHENICELOOKINGPLAFNT
35 : TETCENICELOOKINGPLAFNT
36

THE NICE LOOKING PLAINT will be plausible, and the last three letters will be EXT

In [20]:
block5 = keep_with_keyword("THENICELOOKINGPLAINT", block5)

In [21]:
len(block5)

2

In [22]:
print(block5)

{2: 'TETHENICELOOKINGPLAINT', 6: 'TETHENICELOOKINGPLAINT'}


Luckily, the last block 5 have reduced the candidate to only 2.
Let's compare using them.

In [23]:
answer_candidate = []
for key in block5.keys():
    try:
        print(block3[key]+"___"+block4[key]+"___"+block5[key]+"___")
        answer_candidate.append(key)
    except:
        print(f"{key} is not in block3 or block 4")

INFORMATIONSECURITYARE___MOSTPOPULARKEYWORDSTOC___TETHENICELOOKINGPLAINT___
6 is not in block3 or block 4


Now it's narrowed down to one.

In [24]:
key_22 = built_keys[answer_candidate[0]]
print(key_22)

52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42


It's time to get the last three keys. By using the last_three precomputed in step 2, I'll look for the answer.

In [25]:
final = {}
for i in range(len(last_three)):
    final[i] = key_22 + ' ' + last_three[i]

In [26]:
block5 = {}
for i in range(len(final)):
    plaintext = vc.decrypt(keys=final[i].split(), offset=6, block=5, block_size=25)
    block5[i] = plaintext

In [27]:
block5=keep_with_keyword("THENICELOOKINGPLAINTEXT", block5)
print(block5)

{18: 'TETHENICELOOKINGPLAINTEXT'}


In [28]:
final_key = final[list(block5.keys())[0]]
print(final_key)

52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 34 78 60


## 4. Final Decryption

In [29]:
original = ""
for block in range(1, 6):
    plaintext = vc.decrypt(keys=final_key.split(), offset=6, block=block, block_size=25)
    original += plaintext
print(original)

THISCIPHERWASWIDELYUSEDBECAUSEOFSIMPLESTRUCTURETHEINFORMATIONSECURITYARETHEMOSTPOPULARKEYWORDSTOCREATETHENICELOOKINGPLAINTEXT


In [30]:
print(f"{final_key} /6")

52 19 26 28 02 17 49 38 87 08 94 70 92 88 27 63 39 35 30 31 05 42 34 78 60 /6
